# Generate stackoverflow tag co-occurance matrix for network construction 

In [1]:
pwd()

'/Users/alexandrarottenkolber/Documents/05_Spatial_Inequalities/The-Technology-Space-and-Digital-Development/code_alexandra'

In [2]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from sklearn.preprocessing import OneHotEncoder

In [4]:
count = 0
for chunk in enumerate(pd.read_csv("../../Alexandra/data/stackoverflow.csv", chunksize=1000)):
    count += 1
    if count == 1: 
        df = chunk[1].copy()
    else: 
        break

In [ ]:
df = pd.read_csv("../../Alexandra/data/stackoverflow.csv", low_memory=False)

In [ ]:
type(df)

In [ ]:
df.head()

In [ ]:
no_tag = len(df[df['tags'].apply(lambda x: isinstance(x,float))])
print("There are", no_tag , "observations that do not have tags.")
print("This equals", int(round(no_tag/len(df), 2)*100), "%.")# there are 17362901 observations that do not have tags

In [ ]:
df = df[df['tags'].notna()]
len(df)

In [ ]:
# find unique tags
splitted_list = [string.split("|") for string in list(df["tags"])]
splitted_list_flat = [str_ for sublist in splitted_list for str_ in sublist]
unique_tags = list(set(splitted_list_flat))
unique_ids = list(set(list(df["_id"])))
len(unique_tags), len(unique_ids)

In [ ]:
# how often does each tag show up? 
tag_counts = Counter(splitted_list_flat)

In [ ]:
tag_counts_df = pd.DataFrame.from_dict(tag_counts, orient='index').reset_index().rename(columns = {"index": "tag", 0: "occurence_count"})
tag_counts_df = tag_counts_df.sort_values(by = "occurence_count", ascending=False)
tag_counts_df.head()

In [ ]:
# select top 1000 tags
top_1000_tags_df = tag_counts_df.head(1000).copy()
top_1000_tags_ls = list(top_1000_tags_df["tag"])
top_1000_tags_df.head()

In [ ]:
# keep only frequent tags (top 1000)
splitted_list_filtered = [[x for x in sublist if x in set(top_1000_tags_ls)] for sublist in splitted_list]

In [ ]:
len(splitted_list_filtered)

In [3]:
# read in filtered list 
splitted_list_filtered = []

# open file and read the content in a list
with open(r'../data/splitted_list.txt', 'r') as infile:
    for line in infile:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        splitted_list_filtered.append(x)

In [4]:
len(splitted_list_filtered)
splitted_list_filtered[:10]

["['wpf', 'visual-studio-2010']",
 "['c#', 'visual-studio']",
 "['c#', 'inheritance', 'ef-code-first', 'entity-framework-6']",
 "['c#', 'generics', 'reflection']",
 "['angularjs', 'select', 'angular-ui-router']",
 "['wpf', 'triggers', 'treeview']",
 "['c#', 'winforms', 'button']",
 "['c#', 'css', 'asp.net-mvc-4', 'azure']",
 "['c#', 'inheritance', 'odata']",
 "['c#', 'visual-studio']"]

In [68]:
# Prepare data to calculate co-coocurence matrix 

# 1 Generate one-hot-encodings
# 2 combine column headers and count 
# 3 calculate co-occurance matrix

In [5]:
k = pd.DataFrame(splitted_list_filtered).to_numpy().tolist()

In [6]:
del(splitted_list_filtered)

In [7]:
enc = OneHotEncoder(categories="auto", sparse=False)
enc.fit(k)

OneHotEncoder(sparse=False)

In [8]:
labels=[str(x) for xs in enc.categories_ for x in xs]

In [ ]:
# implementation with sklearn

y = enc.transform(k)#.toarray()
y.shape

In [ ]:
ohe_df = pd.DataFrame(y, columns = labels)

In [ ]:
cnt = Counter(list(ohe_df.columns))
non_unique_labels = [k for k, v in cnt.items() if v > 1 and v != "None" and v != None]
unique_labels = [k for k, v in cnt.items() if v == 1 and v != "None" and v != None]
len(non_unique_labels), len(unique_labels)

In [ ]:
df=ohe_df[unique_labels].copy()
df.shape

In [ ]:
df.head()

In [ ]:
for label in non_unique_labels:
    if label != 'None' and label != None: 
        df[label] = ohe_df[label].sum(axis=1)
        df = df.copy()
df.shape

In [ ]:
tag_df_asint = df.astype(int)
del(df)
coocc = tag_df_asint.T.dot(tag_df_asint)
np.fill_diagonal(coocc.values, 0)
coocc.head()

In [ ]:
# cooccurence matrix to dictionary
coocc_dic = coocc.to_dict()

In [ ]:
# create dataframe represnting a network
tag1_ls = []
frames = []

for tag1, tag_dic in coocc_dic.items():
    tag1_ls.append(tag1)
    frames.append(pd.DataFrame.from_dict(tag_dic, orient='index'))

network_df = pd.concat(frames, keys=tag1_ls)
network_df = network_df.reset_index().rename(columns = {"level_0" : "tag1", "level_1" : "tag2", 0: "cooccurence"})

In [ ]:
network_df[network_df["cooccurence"] > 10].head(10)

In [126]:
# implementation with sklearn

y = OneHotEncoder(categories=unique_tags, 
                  sparse=False, 
                  handle_unknown='error').fit_transform(k[:10]).toarray()
print(y)

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).